-- CLEANING bollywood_movies.sqlite FILE

- importing necessary modules

In [2]:
import sqlite3
import pandas as pd

-- path for my data file

In [3]:
folder="data"
subfolder="movies_data"
filename="bollywood_movies.sqlite"
path=f'{folder}/{subfolder}/{filename}'
path

'data/movies_data/bollywood_movies.sqlite'

-- connect to sqlite DB

In [4]:
conn = sqlite3.connect(path)


- giving a query to check table and read data from file usiing that given query

In [5]:
tables_query = "SELECT name FROM sqlite_master WHERE type='table';"
tables = pd.read_sql(tables_query, conn)
tables

,name
0,Movies


-- giving query to select all data from table "Movies"

In [6]:
query = "SELECT * FROM Movies"
df = pd.read_sql(query, conn)
df.head()

,index,MovieID,Title,Director,Genre,ReleaseYear,Budget (Crores),BoxOffice (Crores),Rating,Duration (minutes),LeadActor,LeadActress,Language,ProductionCompany
0,0,MOV001,Lagaan,Ashutosh Gowariker,None,2001,NaN,100.0,8.1,224,Aamir Khan,Gracy Singh,Hindi,Aamir Khan Productions
1,1,MOV002,Kabhi Khushi Kabhie Gham...,Karan Johar,Family Drama,2001,50.0,100.0,7.9,210,Shah Rukh Khan,Kajol,Hindi,Dharma Productions
2,2,MOV003,M.S. Dhoni: The Untold Story,Neeraj Pandey,Biographical Sports Drama,2016,NaN,215.0,7.9,184,Sushant Singh Rajput,Kiara Advani,Hindi,Fox Star Studios
3,3,MOV004,Baahubali 2: The Conclusion,S. S. Rajamouli,Epic Fantasy Action,2017,250.0,1810.0,8.2,171,Prabhas,Anushka Shetty,Telugu,Arka Media Works
4,4,MOV005,Chennai Express,Rohit Shetty,Action Comedy,2013,NaN,423.0,6.4,141,Shah Rukh Khan,Deepika Padukone,Hindi,Red Chillies Entertainment


-- close connnectiion after storing the data in df for avoiding memory leakage

In [7]:
conn.close

<function Connection.close()>

-- Now we have our data in dataframe, we can check for anomalies and clean it

In [8]:
df.columns

Index(['index', 'MovieID', 'Title', 'Director', 'Genre', 'ReleaseYear',
       'Budget (Crores)', 'BoxOffice (Crores)', 'Rating', 'Duration (minutes)',
       'LeadActor', 'LeadActress', 'Language', 'ProductionCompany'],
      dtype='object')

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55 entries, 0 to 54
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   index               55 non-null     int64  
 1   MovieID             55 non-null     object 
 2   Title               55 non-null     object 
 3   Director            55 non-null     object 
 4   Genre               54 non-null     object 
 5   ReleaseYear         55 non-null     int64  
 6   Budget (Crores)     43 non-null     float64
 7   BoxOffice (Crores)  46 non-null     float64
 8   Rating              55 non-null     float64
 9   Duration (minutes)  55 non-null     int64  
 10  LeadActor           55 non-null     object 
 11  LeadActress         55 non-null     object 
 12  Language            55 non-null     object 
 13  ProductionCompany   55 non-null     object 
dtypes: float64(3), int64(3), object(8)
memory usage: 6.1+ KB


-- missing data in Genre column

In [10]:
df.Genre.unique()

array([None, 'Family Drama', 'Biographical Sports Drama',
       'Epic Fantasy Action', 'Action Comedy', 'Drama', 'Action Thriller',
       'Satirical Science Fiction Comedy', 'Science Fiction Action',
       'Historical Romance', 'Romance', 'Superhero Science Fiction',
       'Action-Adventure', 'Musical Drama', 'Historical Action',
       'Action Drama', 'Superhero', 'Sports Drama', 'Sports Action',
       'Romantic Comedy', 'Romantic Drama', 'Thriller',
       'Psychological Thriller', 'Historical Sports Drama',
       'Comedy Drama', 'Biographical Drama', 'Satirical Comedy'],
      dtype=object)

-- we can fill missing value with most common genre

In [11]:
df["Genre"].mode()[0]  

'Action Thriller'

-- checking the other columns like Title for row with none value in genre

In [12]:
df[df["Genre"].isna()]

,index,MovieID,Title,Director,Genre,ReleaseYear,Budget (Crores),BoxOffice (Crores),Rating,Duration (minutes),LeadActor,LeadActress,Language,ProductionCompany
0,0,MOV001,Lagaan,Ashutosh Gowariker,None,2001,NaN,100.0,8.1,224,Aamir Khan,Gracy Singh,Hindi,Aamir Khan Productions


-- Movie with none value of genre is Lagaan , Most frequent genre is Action thriller which isn't appropriate genre for this movie So I am filling this with correct( more appropriate genre ) value instead

In [13]:
df["Genre"]=df["Genre"].fillna("Historical Sports Drama")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55 entries, 0 to 54
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   index               55 non-null     int64  
 1   MovieID             55 non-null     object 
 2   Title               55 non-null     object 
 3   Director            55 non-null     object 
 4   Genre               55 non-null     object 
 5   ReleaseYear         55 non-null     int64  
 6   Budget (Crores)     43 non-null     float64
 7   BoxOffice (Crores)  46 non-null     float64
 8   Rating              55 non-null     float64
 9   Duration (minutes)  55 non-null     int64  
 10  LeadActor           55 non-null     object 
 11  LeadActress         55 non-null     object 
 12  Language            55 non-null     object 
 13  ProductionCompany   55 non-null     object 
dtypes: float64(3), int64(3), object(8)
memory usage: 6.1+ KB


In [14]:
df[df['Budget (Crores)'].isna()]

,index,MovieID,Title,Director,Genre,ReleaseYear,Budget (Crores),BoxOffice (Crores),Rating,Duration (minutes),LeadActor,LeadActress,Language,ProductionCompany
0,0,MOV001,Lagaan,Ashutosh Gowariker,Historical Sports Drama,2001,NaN,100.0,8.1,224,Aamir Khan,Gracy Singh,Hindi,Aamir Khan Productions
2,2,MOV003,M.S. Dhoni: The Untold Story,Neeraj Pandey,Biographical Sports Drama,2016,NaN,215.0,7.9,184,Sushant Singh Rajput,Kiara Advani,Hindi,Fox Star Studios
4,4,MOV005,Chennai Express,Rohit Shetty,Action Comedy,2013,NaN,423.0,6.4,141,Shah Rukh Khan,Deepika Padukone,Hindi,Red Chillies Entertainment
7,7,MOV008,Sarkar (Tamil),A. R. Murugadoss,Action Thriller,2005,NaN,NaN,7.5,173,Vijay,Keerthy Suresh,Tamil,Kalaignar TV
11,11,MOV012,Dilwale Dulhania Le Jayenge,Aditya Chopra,Romance,1995,NaN,NaN,7.7,189,Shah Rukh Khan,Kajol,Hindi,Yash Raj Films
23,23,MOV024,Avengers: Endgame (Dubbed),"Anthony Russo, Joseph Russo",Superhero,2019,NaN,NaN,8.4,181,Robert Downey Jr.,Scarlett Johansson,"English (Dubbed in Telugu, Kannada, Hindi)",Marvel Studios
32,32,MOV033,96 (Kannada Dubbed),C. Premkumar,Romantic Drama,2019,NaN,NaN,7.9,158,Vijay Sethupathi,Trisha,Kannada (Dubbed from Tamil),Madras Enterprises
34,34,MOV035,Dabangg 3 (Dubbed),Prabhu Deva,Action Comedy,2019,NaN,NaN,5.8,159,Salman Khan,Sonakshi Sinha,Telugu (Dubbed from Hindi),S. K. Film Enterprises
36,36,MOV037,Shylock (Kannada Dubbed),Ajai Vasudev,Action Thriller,2020,NaN,NaN,6.8,164,Mammootty,Rajkiran,Kannada (Dubbed from Malayalam),Vrindaavan Films
37,37,MOV038,Love Aaj Kal (Dubbed),Imtiaz Ali,Romantic Drama,2020,NaN,NaN,4.4,141,Kartik Aaryan,Sara Ali Khan,Telugu (Dubbed from Hindi),Reliance Entertainment


-- Handling missing values in Budget(Crores) and BoxOffice (Crores) . Since they are numerical columns I'm filling null values with median

In [15]:
df['Budget (Crores)']=df['Budget (Crores)'].fillna(df["Budget (Crores)"].median())
df['BoxOffice (Crores)']=df['BoxOffice (Crores)'].fillna(df['BoxOffice (Crores)'].median())
cols=['Budget (Crores)','BoxOffice (Crores)']
df[cols]

,Budget (Crores),BoxOffice (Crores)
0,50.0,100.0
1,50.0,100.0
2,50.0,215.0
3,250.0,1810.0
4,50.0,423.0
5,100.0,325.0
6,70.0,2140.0
7,50.0,250.0
8,85.0,792.0
9,550.0,800.0


In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55 entries, 0 to 54
Data columns (total 14 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   index               55 non-null     int64  
 1   MovieID             55 non-null     object 
 2   Title               55 non-null     object 
 3   Director            55 non-null     object 
 4   Genre               55 non-null     object 
 5   ReleaseYear         55 non-null     int64  
 6   Budget (Crores)     55 non-null     float64
 7   BoxOffice (Crores)  55 non-null     float64
 8   Rating              55 non-null     float64
 9   Duration (minutes)  55 non-null     int64  
 10  LeadActor           55 non-null     object 
 11  LeadActress         55 non-null     object 
 12  Language            55 non-null     object 
 13  ProductionCompany   55 non-null     object 
dtypes: float64(3), int64(3), object(8)
memory usage: 6.1+ KB


-- Our data is cleaned since all columns now contain non null 55 rows and they have correct datatype

-- Now Store the cleaned df into excel file

In [17]:
df.to_excel('bollywood_movies.xlsx', index=False)

-- Getting info for other tables

- ReleaseYear for MOV049

In [18]:
df[df['MovieID']=='MOV049']

,index,MovieID,Title,Director,Genre,ReleaseYear,Budget (Crores),BoxOffice (Crores),Rating,Duration (minutes),LeadActor,LeadActress,Language,ProductionCompany
48,48,MOV049,U Turn,Pawan Kumar,Thriller,2016,5.0,20.0,7.9,128,Shraddha Srinath,Dilip Raj,Kannada,PRK Productions


-- Budget and BoxOffice info for MOV008, MOV024, MOV033, MOV035, MOV037, MOV038, MOV039, MOV040

In [19]:
rows=['MOV008', 'MOV024', 'MOV033', 'MOV035', 'MOV037', 'MOV038', 'MOV039', 'MOV040']
df[df['MovieID'].isin(rows)]

,index,MovieID,Title,Director,Genre,ReleaseYear,Budget (Crores),BoxOffice (Crores),Rating,Duration (minutes),LeadActor,LeadActress,Language,ProductionCompany
7,7,MOV008,Sarkar (Tamil),A. R. Murugadoss,Action Thriller,2005,50.0,250.0,7.5,173,Vijay,Keerthy Suresh,Tamil,Kalaignar TV
23,23,MOV024,Avengers: Endgame (Dubbed),"Anthony Russo, Joseph Russo",Superhero,2019,50.0,250.0,8.4,181,Robert Downey Jr.,Scarlett Johansson,"English (Dubbed in Telugu, Kannada, Hindi)",Marvel Studios
32,32,MOV033,96 (Kannada Dubbed),C. Premkumar,Romantic Drama,2019,50.0,250.0,7.9,158,Vijay Sethupathi,Trisha,Kannada (Dubbed from Tamil),Madras Enterprises
34,34,MOV035,Dabangg 3 (Dubbed),Prabhu Deva,Action Comedy,2019,50.0,250.0,5.8,159,Salman Khan,Sonakshi Sinha,Telugu (Dubbed from Hindi),S. K. Film Enterprises
36,36,MOV037,Shylock (Kannada Dubbed),Ajai Vasudev,Action Thriller,2020,50.0,250.0,6.8,164,Mammootty,Rajkiran,Kannada (Dubbed from Malayalam),Vrindaavan Films
37,37,MOV038,Love Aaj Kal (Dubbed),Imtiaz Ali,Romantic Drama,2020,50.0,250.0,4.4,141,Kartik Aaryan,Sara Ali Khan,Telugu (Dubbed from Hindi),Reliance Entertainment
38,38,MOV039,Forensic (Kannada Dubbed),"Akhil Paul, Anas Khan",Thriller,2020,50.0,250.0,6.9,129,Tovino Thomas,Mamta Mohandas,Kannada (Dubbed from Malayalam),Evil Eye Pictures
39,39,MOV040,Trance (Kannada Dubbed),Anwar Rasheed,Psychological Thriller,2020,50.0,250.0,7.1,162,Fahadh Faasil,Nazriya Nazim,Kannada (Dubbed from Malayalam),Dulquer Salmaan Productions


-- getting the value of Rating for movie with id MOV031 to fill missing data in indian_movies.csv file

In [20]:
df[df['MovieID']=='MOV031']

,index,MovieID,Title,Director,Genre,ReleaseYear,Budget (Crores),BoxOffice (Crores),Rating,Duration (minutes),LeadActor,LeadActress,Language,ProductionCompany
30,30,MOV031,Saaho,Sujeeth,Action Thriller,2019,300.0,450.0,6.5,170,Prabhas,Shraddha Kapoor,"Telugu (Dubbed in Hindi, Kannada)",UV Creations


-- gettig duration value for movie with id MOV026 to update missing value in indian_movies table

In [21]:
df[df['MovieID']=='MOV026']

,index,MovieID,Title,Director,Genre,ReleaseYear,Budget (Crores),BoxOffice (Crores),Rating,Duration (minutes),LeadActor,LeadActress,Language,ProductionCompany
25,25,MOV026,Pailwaan,S. Krishna,Sports Action,2019,30.0,55.0,6.8,166,Sudeep,Aakanksha Singh,Kannada,RRR Motion Pictures


-- Finding values for updating the tollywood_movies.xlsx file with correct value wherever there is missing data

-- getting ReleaseYear for MOV034 and MOV004

In [22]:
rows=['MOV004','MOV034']
df[df['MovieID'].isin(rows)]

,index,MovieID,Title,Director,Genre,ReleaseYear,Budget (Crores),BoxOffice (Crores),Rating,Duration (minutes),LeadActor,LeadActress,Language,ProductionCompany
3,3,MOV004,Baahubali 2: The Conclusion,S. S. Rajamouli,Epic Fantasy Action,2017,250.0,1810.0,8.2,171,Prabhas,Anushka Shetty,Telugu,Arka Media Works
33,33,MOV034,Sarileru Neekevvaru,Anil Ravipudi,Action Comedy,2020,75.0,260.0,7.1,169,Mahesh Babu,Rashmika Mandanna,Telugu,AK Entertainments


-- extracting record with movieID MOV027 for updating missing rating value in tollywood_movies data

In [23]:
df[df['MovieID']=='MOV027']

,index,MovieID,Title,Director,Genre,ReleaseYear,Budget (Crores),BoxOffice (Crores),Rating,Duration (minutes),LeadActor,LeadActress,Language,ProductionCompany
26,26,MOV027,Geetha Govindam,Parasuram,Romantic Comedy,2018,10.0,130.0,7.4,148,Vijay Deverakonda,Rashmika Mandanna,Telugu,GA2 Pictures


-- extracting record with movieID MOV025 for updating missing Duration value in tollywood_movies data

In [24]:
df[df['MovieID']=='MOV027']

,index,MovieID,Title,Director,Genre,ReleaseYear,Budget (Crores),BoxOffice (Crores),Rating,Duration (minutes),LeadActor,LeadActress,Language,ProductionCompany
26,26,MOV027,Geetha Govindam,Parasuram,Romantic Comedy,2018,10.0,130.0,7.4,148,Vijay Deverakonda,Rashmika Mandanna,Telugu,GA2 Pictures


-- extracting records with movieIDs MOV025, MOV027, MOV029 to fill missing Language values in tollywood_movies table

In [25]:
rows=['MOV025','MOV027','MOV029']
df[df['MovieID'].isin(rows)]

,index,MovieID,Title,Director,Genre,ReleaseYear,Budget (Crores),BoxOffice (Crores),Rating,Duration (minutes),LeadActor,LeadActress,Language,ProductionCompany
24,24,MOV025,Jersey,Gowtam Tinnanuri,Sports Drama,2019,20.0,45.0,7.8,160,Nani,Shraddha Srinath,Telugu,Sithara Entertainments
26,26,MOV027,Geetha Govindam,Parasuram,Romantic Comedy,2018,10.0,130.0,7.4,148,Vijay Deverakonda,Rashmika Mandanna,Telugu,GA2 Pictures
28,28,MOV029,Dear Comrade,Bharat Kamma,Romantic Drama,2019,15.0,35.0,7.1,170,Vijay Deverakonda,Rashmika Mandanna,Telugu,Mythri Movie Makers


-- getting record with moviedID MOV021 to update tollywood_movies table with correct value of ProductionCompany

In [26]:
df[df['MovieID']=='MOV021']

,index,MovieID,Title,Director,Genre,ReleaseYear,Budget (Crores),BoxOffice (Crores),Rating,Duration (minutes),LeadActor,LeadActress,Language,ProductionCompany
20,20,MOV021,Bahubali: The Beginning,S. S. Rajamouli,Epic Fantasy Action,2015,180.0,650.0,8.1,159,Prabhas,Tamannaah,Telugu,Arka Media Works
